Searching Api: My audience is exigent, they are teenagers. I must serve the best.

__Project Web__<br>

__by Jordi Marsal__<br><br>

# - Trying a Cool API<br>
Or: "Molar muy fuerte, pregúntame como"

## Marvel API<br>
### developper account:
#### https://developer.marvel.com/account
__Rate Limits:__<br>
3000 calls/day<br><br>
__Attribution:__<br>
Data provided by Marvel. © 2019 Marvel<br><br>
__Linking to Marvel:__<br>
Linking from your application and web pages to Marvel helps us understand how applications are being used by end-users, which applications are most successful, and which presentations of Marvel content is most impactful, all of which helps us improve our API products.<br><br>
If a URL array is not present for an entity simply link to the Marvel.com home page (http://marvel.com) in your attribution notice.<br><br>

GENERAL API INFORMATION<br><br>
The Marvel Comics __API__ is a __RESTful__ service which provides methods for accessing specific resources at canonical URLs and for searching and filtering sets of resources by various criteria. All representations are encoded as __JSON__ objects.


## Giphy API<br>
### developper account:
#### https://developers.giphy.com
Create an App<br>
Implement Search<br>
Incorporate Analytics<br>
Attribution<br>


### Imports

In [1]:
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install
#!jupyter nbextensions_configurator enable --user

In [2]:
#!pip install requests
#!pip install bs4
#!pip install jupyter_dashboards
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
from functools import reduce
from datetime import datetime
#import hashlib
from bs4 import BeautifulSoup 
import re
import random
import urllib.parse
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, IFrame, HTML
import jupyter_dashboards


### Marvel API Info 

In [3]:
struct = {
    'endpoint':'http://gateway.marvel.com:80',
    'characters': '/v1/public/characters',
    'comics':'/v1/public/comics',
    'creators':'/v1/public/creators',
    'events':'/v1/public/events',
    'series':'/v1/public/series',
    'stories':'/v1/public/stories',
    'pcharacters':['name','nameStartsWith','limit','modifiedSince','comics','events','series','stories']
}
call = 'https://gateway.marvel.com:443/v1/public/characters?nameStartsWith=Spider&limit=25&apikey=e27f579a21906a26a77b3e6fce0cc65d'


### Classes: Aux / APIMarvel

In [4]:
class Aux:
    INFO = 0
    DEBUG = 1
    WARN = 2
    ERROR = 3
    NONE = 4
    def timestamp():
        now = datetime.now()
        return ''.join(str(datetime.timestamp(now)).split('.'))

    def string(string, var):
        return string % var
    
    def get_state(res, level=4):
        if level <= Aux.DEBUG:
            print("## Status code = "+str(res.status_code))
            print("## Message = "+str(res.text))
        if res.status_code >=200 and res.status_code <300:
            print("  This is OK")
        if res.status_code >=300 and res.status_code <400:
            print("  This is a redirection")
        if res.status_code >=400 and res.status_code <500:
            print("  This is a Client problem")
        if res.status_code >=500 and res.status_code <600:
            print("  This is a Server problem")
        if level == Aux.INFO:
            print("#HEADERS#", res.headers)
            print("#CONTENT#", res.content)
            print("#JSON#", res.json())
            
    #hash - a md5 digest of the ts parameter, your private key and your public key (e.g. md5(ts+privateKey+publicKey)
    def hash_md5(string):
        return hashlib.md5(string.encode('utf-8')).hexdigest()
    
    def image(url):
        return Image(url, width=100, height=100)

class APIMarvel:
    def __init__(self, struct, api_key, priv_key):
        self.struct = struct
        self.endpoint = struct['endpoint']
        self.end = '/'
        self.api_key = api_key
        self.priv_key = priv_key
        self.err = []
        self.headers = {'Accept': '*/*'}
    
    def forced(self, params):
        if 'limit' not in params:
            params['limit']='10'
        t = Aux.timestamp()
        params['ts'] = t
        params['apikey'] = self.api_key
        params['hash'] = Aux.hash_md5(t+self.priv_key+self.api_key)
        print(params)
        return params
    
    def _validate(self, path, params):
        if path not in self.struct:
            self.err.append(Aux.string("Path %s not exists",path))
        if not self.err:
            pset = "%s%s" % ('p', path)
            pparams = self.struct[pset]
            for p in params:
                if p not in pparams:
                    self.err.append(Aux.string("Parameter %s not exists",p))
        if self.err:
            print(self.err)
            
        return len(self.err)==0
    
    def _get_url(self, path, params):
        url = self.endpoint + self.struct[path]
        params = self.forced(params)
        if params:
            url += "?"
            for i,(key, value) in enumerate(params.items()):
                end = '$' if i < len(params) -1 else ''
                url += "%s=%s%s" % (key, value, end)
        
        print('URL:',url)
        return url
            
    def _connect(self, url):
        headers = self.get_random_ua()
        return requests.get(url, headers = headers)
        
    def get_json(self, url):
        response = self._connect(url)
        Aux.get_state(response)
        return self._connect(url).json()
    
    def get(self, path, params={}):
        if self._validate(path, params):
            self.get_json(self._get_url(path,params))
            
    def clear_err(self):
        self.err = []
        
    def get_random_ua(self):
        heads =[
            'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:67.0) Gecko/20100101 Firefox/67.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15'
        ]
        headers = {'User-Agent': '','Accept': '*/*'}
        sample = random.sample(heads,1)
        headers['User-Agent'] = sample[0]
        return headers

### Giphy API Info

In [5]:

struct = {
    'endpoint':'http://api.giphy.com/v1/gifs/search',
    'api_key': 'XXX',
    'search': 'http://api.giphy.com/v1/gifs/search',
    'psearch':['t','q','limit','offset','rating','lang','random_id']
}

### Class APIGiphy 

In [6]:
class APIGiphy:
    limit = 5
    def __init__(self, struct):
        self.struct = struct
        self.endpoint = struct['endpoint']
        self.end = '/'
        self.api_key = struct['api_key']
        self.err = []
        self.debug_level = Aux.DEBUG
    
    def forced(self, params):
        params['api_key'] = self.api_key
        if 'limit' not in params:
            params['limit'] = str(APIGiphy.limit)
        if self.debug_level <= Aux.DEBUG:
            print(params)
        return params
    
    def _validate(self, path, params):
        if path not in self.struct:
            self.err.append(Aux.string("Path %s not exists",path))
        if not self.err:
            pset = "%s%s" % ('p', path)
            pparams = self.struct[pset]
            for p in params:
                if p not in pparams:
                    self.err.append(Aux.string("Parameter %s not exists",p))
        if self.err:
            print(self.err)
            
        return len(self.err)==0
    
    def _get_url(self, path, params):
        url = self.struct[path]
        params = self.forced(params)
        if params:
            url += "?"
            for i,(key, value) in enumerate(params.items()):
                end = '&' if i < len(params) -1 else ''
                url += "%s=%s%s" % (key, value, end)
        
        print('URL:',url)
        return url
            
    def _connect(self, url):
        headers = self.get_random_ua()
        return requests.get(url, headers = headers)
        
    def get_json(self, url,level=4):
        response = self._connect(url)
        Aux.get_state(response,level)
        return response.json()
    
    def get(self, path, params={},level=Aux.NONE):
        if self._validate(path, params):
            return self.get_json(self._get_url(path,params),level)
            
    def clear_err(self):
        self.err = []
        
    def get_random_ua(self):
        heads =[
            'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:67.0) Gecko/20100101 Firefox/67.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15'
        ]
        headers = {'User-Agent': '','Accept': '*/*'}
        sample = random.sample(heads,1)
        headers['User-Agent'] = sample[0]
        return headers

In [7]:
api = APIGiphy(struct)

In [8]:
#json = api.get('characters')
#df = pd.DataFrame(json)
#df.head()
'''
omdb = 'http://www.omdbapi.com/?apikey=f2f47e82&t=Spider&y=2019'

ojson = api.get_json(omdb)

ordb = pd.DataFrame(ojson)
print(ojson)
ordb
'''

"\nomdb = 'http://www.omdbapi.com/?apikey=f2f47e82&t=Spider&y=2019'\n\nojson = api.get_json(omdb)\n\nordb = pd.DataFrame(ojson)\nprint(ojson)\nordb\n"

# Web Scrapping

## Wikipedia: MCU

In [9]:
url_iframe = 'https://es.wikipedia.org/wiki/Universo_cinematogr%C3%A1fico_de_Marvel#Pel%C3%ADculas'
IFrame(url_iframe,width=1000,height=400)

### Request

In [10]:
url = 'https://es.wikipedia.org/wiki/Universo_cinematogr%C3%A1fico_de_Marvel'
html = requests.get(url).content
html

b'<!DOCTYPE html>\n<html class="client-nojs" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Universo cinematogr\xc3\xa1fico de Marvel - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Universo_cinematogr\xc3\xa1fico_de_Marvel","wgTitle":"Universo cinematogr\xc3\xa1fico de Marvel","wgCurRevisionId":117689759,"wgRevisionId":117689759,"wgArticleId":2830688,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:P\xc3\xa1ginas con errores de referencia","Wikipedia:P\xc3\xa1ginas con plantillas con argumentos duplicados","Wikipedia:P\xc3\xa1ginas con referencias con par\xc3\xa1metros sugeridos (URL)","Wikipedia:P\xc3\xa1ginas con referencias web sin URL","Wikipedia:P\xc3\xa1ginas con referen

### Scrap with Beautiful Soup

In [11]:
soup = BeautifulSoup(html,'lxml')
table = soup.find_all('table',{'class':'wikitable plainrowheaders'})[0]
table

<table border="1" class="wikitable plainrowheaders" style="text-align:center" width="100%">
<tbody><tr>
<th scope="col">Película
</th>
<th scope="col">Estreno en EE.UU.
</th>
<th scope="col">Director(es)
</th>
<th scope="col">Guionista(s)
</th>
<th scope="col">Productor(es)
</th>
<th scope="col">Estado
</th>
<th scope="col"><abbr title="Referencias">Ref.</abbr>
</th></tr>
<tr>
<th colspan="7" style="background-color: #B21A27;color:white">La saga del infinito
</th></tr>
<tr>
<th colspan="7" style="background-color: #E62E00;color:white">Fase uno
</th></tr>
<tr>
<th scope="row" style="text-align:left"><i><a href="/wiki/Iron_Man_(pel%C3%ADcula)" title="Iron Man (película)">Iron Man</a></i>
</th>
<td style="text-align:left">2 de mayo de 2008
</td>
<td><a href="/wiki/Jon_Favreau" title="Jon Favreau">Jon Favreau</a>
</td>
<td>Mark Fergus, Hawk Ostby, Art Marcum y Matt Holloway
</td>
<td><a href="/wiki/Avi_Arad" title="Avi Arad">Avi Arad</a> y <a href="/wiki/Kevin_Feige" title="Kevin Feige">Ke

### Cleaning with regex and List Comprehensions

In [12]:
def no_brackets(string):
    r = re.sub('\[\d+\]', '',string)
    return r

rows = table.find_all('tr')

films = [no_brackets(span.a.text) for span in rows if span.a and no_brackets(span.a.text)]

print (films)

['Iron Man', 'The Incredible Hulk', 'Iron Man 2', 'Thor', 'Capitán América: el primer vengador', 'The Avengers', 'Iron Man 3', 'Thor: The Dark World', 'Captain America: The Winter Soldier', 'Guardianes de la Galaxia', 'Avengers: Age of Ultron', 'Ant-Man', 'Capitán América: Civil War', 'Doctor Strange', 'Guardianes de la Galaxia Vol. 2', 'Spider-Man: Homecoming', 'Thor: Ragnarok', 'Black Panther', 'Avengers: Infinity War', 'Ant-Man and the Wasp', 'Capitana Marvel', 'Avengers: Endgame', 'Spider-Man: Lejos de casa', 'Black Widow', 'Eternals']


# Integrating All
### OMDB Try

In [13]:

def get_film(title):
    base_url = 'XXX'
    url = base_url+"&t="+title
    print(url)
    fjson = api.get_json(url,Aux.INFO)

dfb = pd.DataFrame()
for t in films[:1]:
    print(t)
    fjson = get_film(t)
    print(fjson)


Iron Man
http://www.omdbapi.com?apikey=f2f47e82&t=Iron Man
## Status code = 200
## Message = {"Title":"Iron Man","Year":"2008","Rated":"PG-13","Released":"02 May 2008","Runtime":"126 min","Genre":"Action, Adventure, Sci-Fi","Director":"Jon Favreau","Writer":"Mark Fergus (screenplay), Hawk Ostby (screenplay), Art Marcum (screenplay), Matt Holloway (screenplay), Stan Lee (characters), Don Heck (characters), Larry Lieber (characters), Jack Kirby (characters)","Actors":"Robert Downey Jr., Terrence Howard, Jeff Bridges, Gwyneth Paltrow","Plot":"After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.","Language":"English, Persian, Urdu, Arabic, Hungarian","Country":"USA","Awards":"Nominated for 2 Oscars. Another 20 wins & 65 nominations.","Poster":"https://m.media-amazon.com/images/M/MV5BMTczNTI2ODUwOF5BMl5BanBnXkFtZTcwMTU0NTIzMw@@._V1_SX300.jpg","Ratings":[{"Source":"Internet Movie Database","Value":"7.9/10"},{"Sou

## Giphy + web results Integration

In [14]:
def get_gif(title):
    return api.get('search',{'q':urllib.parse.quote(title)},level=Aux.NONE)


for i,t in enumerate(films):
    print('Film:',t)
    fjson = get_gif(t)
    if i==0:
        df1 = pd.DataFrame(json_normalize(fjson['data']))
    else:
        df = pd.DataFrame(json_normalize(fjson['data']))
        df1 = pd.concat([df1, df], axis='rows')
    
df1 = df1.reset_index(drop=True)


Film: Iron Man
{'q': 'Iron%20Man', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=Iron%20Man&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Film: The Incredible Hulk
{'q': 'The%20Incredible%20Hulk', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=The%20Incredible%20Hulk&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Film: Iron Man 2
{'q': 'Iron%20Man%202', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=Iron%20Man%202&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


  This is OK
Film: Thor
{'q': 'Thor', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=Thor&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Film: Capitán América: el primer vengador
{'q': 'Capit%C3%A1n%20Am%C3%A9rica%3A%20el%20primer%20vengador', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=Capit%C3%A1n%20Am%C3%A9rica%3A%20el%20primer%20vengador&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Film: The Avengers
{'q': 'The%20Avengers', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=The%20Avengers&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Film: Iron Man 3
{'q': 'Iron%20Man%203', 'api_key': 'qalYklnRNsDnfcN8yepwUSH8NfIX4SqL', 'limit': '5'}
URL: http://api.giphy.com/v1/gifs/search?q=Iron%20Man%203&api_key=qalYklnRNsDnfcN8yepwUSH8NfIX4SqL&limit=5
  This is OK
Film: Thor: The 

### subset of urls

In [15]:
dfgif = df1[['title','images.downsized_medium.url','images.preview_gif.url','embed_url']]
dfgif

,title,images.downsized_medium.url,images.preview_gif.url,embed_url
0,suit up iron man GIF by 100% Soft,https://media0.giphy.com/media/l3975CZuyQgoNVu...,https://media0.giphy.com/media/l3975CZuyQgoNVu...,https://giphy.com/embed/l3975CZuyQgoNVuOA
1,iron man eye roll GIF,https://media2.giphy.com/media/qmfpjpAT2fJRK/g...,https://media2.giphy.com/media/qmfpjpAT2fJRK/g...,https://giphy.com/embed/qmfpjpAT2fJRK
2,iron man GIF,https://media1.giphy.com/media/VFB3cJJne7b5m/g...,https://media1.giphy.com/media/VFB3cJJne7b5m/g...,https://giphy.com/embed/VFB3cJJne7b5m
3,iron man bathroom GIF,https://media0.giphy.com/media/iHTRo3hYZGhZS/g...,https://media0.giphy.com/media/iHTRo3hYZGhZS/g...,https://giphy.com/embed/iHTRo3hYZGhZS
4,iron man eye roll GIF,https://media2.giphy.com/media/RQMkfKODe8Okw/g...,https://media2.giphy.com/media/RQMkfKODe8Okw/g...,https://giphy.com/embed/RQMkfKODe8Okw
5,weird see ya GIF by Cheezburger,https://media3.giphy.com/media/XKL1MHvk47OXC/g...,https://media3.giphy.com/media/XKL1MHvk47OXC/g...,https://giphy.com/embed/XKL1MHvk47OXC
6,the hulk smiling GIF,https://media2.giphy.com/media/Gwtv1sXXnunMA/g...,https://media2.giphy.com/media/Gwtv1sXXnunMA/g...,https://giphy.com/embed/Gwtv1sXXnunMA
7,dc comics film GIF,https://media2.giphy.com/media/1zv2iEutcQAqA/g...,https://media2.giphy.com/media/1zv2iEutcQAqA/g...,https://giphy.com/embed/1zv2iEutcQAqA
8,excited hulk smash GIF,https://media2.giphy.com/media/3aUGq69kSLj4k/g...,https://media2.giphy.com/media/3aUGq69kSLj4k/g...,https://giphy.com/embed/3aUGq69kSLj4k
9,hulk smash GIF,https://media1.giphy.com/media/UrfLgfuQG41sQ/g...,https://media1.giphy.com/media/UrfLgfuQG41sQ/g...,https://giphy.com/embed/UrfLgfuQG41sQ


### Some Gifs

![images.preview_gif.url](https://media2.giphy.com/media/UJ5I7921pAOEU/giphy-preview.gif?cid=a937cb845d34786d533447463627d59c&rid=giphy-preview.gif
)

In [16]:
#mini = images.preview_gif.url
#medium= images.downsized_medium.url
# images.downsized_medium.url
url = 'https://giphy.com/embed/UJ5I7921pAOEU'
url2 =  df1['embed_url'][0]
print(url2)
#IFrame(src=url, width=350, height=200)
IFrame(src=url, width=350, height=200)

https://giphy.com/embed/l3975CZuyQgoNVuOA


### Dictionary for final URLs

In [17]:
dic = {}
def create_dic():
    for i,ti in enumerate(films):
        dic[ti]= list(dfgif.iloc[i*5:i*5+5]['images.downsized_medium.url'])

create_dic()
print(dic)

'''
def uniques_values_all(serie):
    uni =  serie.unique().tolist()
    return uni
titles = uniques_values_all(df1.title)
urls = uniques_values_all(df1['images.downsized_medium.url'])
titles
'''

{'Iron Man': ['https://media0.giphy.com/media/l3975CZuyQgoNVuOA/giphy.gif?cid=a937cb845d39d4bb4c2e575632d14954&rid=giphy.gif', 'https://media2.giphy.com/media/qmfpjpAT2fJRK/giphy.gif?cid=a937cb845d39d4bb4c2e575632d14954&rid=giphy.gif', 'https://media1.giphy.com/media/VFB3cJJne7b5m/giphy.gif?cid=a937cb845d39d4bb4c2e575632d14954&rid=giphy.gif', 'https://media0.giphy.com/media/iHTRo3hYZGhZS/giphy-downsized-medium.gif?cid=a937cb845d39d4bb4c2e575632d14954&rid=giphy-downsized-medium.gif', 'https://media2.giphy.com/media/RQMkfKODe8Okw/giphy.gif?cid=a937cb845d39d4bb4c2e575632d14954&rid=giphy.gif'], 'The Incredible Hulk': ['https://media3.giphy.com/media/XKL1MHvk47OXC/giphy.gif?cid=a937cb845d39d4bc5477487655acccc6&rid=giphy.gif', 'https://media2.giphy.com/media/Gwtv1sXXnunMA/giphy.gif?cid=a937cb845d39d4bc5477487655acccc6&rid=giphy.gif', 'https://media2.giphy.com/media/1zv2iEutcQAqA/giphy.gif?cid=a937cb845d39d4bc5477487655acccc6&rid=giphy.gif', 'https://media2.giphy.com/media/3aUGq69kSLj4k/giphy

"\ndef uniques_values_all(serie):\n    uni =  serie.unique().tolist()\n    return uni\ntitles = uniques_values_all(df1.title)\nurls = uniques_values_all(df1['images.downsized_medium.url'])\ntitles\n"

### Set Widgets

In [18]:

dropd_gif = widgets.Dropdown(options = films)
output = widgets.Output()

In [19]:
def df_filtering(gif,film_name):
    output.clear_output()
    urls = dic[film_name]
    with output:
        display(HTML("""<div style=\"display:inline-flex\">
        <img height=\"100\" src='%s'>
        &nbsp;
        <img style=\"margin: 0;\" height=\"100\" src='%s'></div>
        <div style=\"display:inline-flex\">
        <img height=\"100\" src='%s'>
        &nbsp;
        <img style=\"margin: 0;\" height=\"100\" src='%s'></div>""" %(urls[0],urls[1],urls[2],urls[3])))

![images.preview_gif.url](url
)

In [20]:
def dr_gif_handler(change):
    df_filtering(change.new,dropd_gif.value)
    
dropd_gif.observe(dr_gif_handler,names='value')
    
display(dropd_gif)

Dropdown(options=('Iron Man', 'The Incredible Hulk', 'Iron Man 2', 'Thor', 'Capitán América: el primer vengado…

## Output

In [21]:
display(output)

Output()